In [3]:
import pandas as pd

In [4]:
file =open('sample','r', errors='ignore')
file2 =open('replaced','w+')
contents=file.read()
for x in range(len(contents)):

    if contents[x] == ",":
        file2.write("!!@@@")
    else:
        file2.write(str(contents[x]))
file2.close()

In [5]:
names=["Month" ,"Day" ,"Time" , "Machine", "State"]
df= pd.read_csv('replaced', sep=" " , usecols=range(5) ,names=names)

In [6]:
df["State"] = df["State"].replace({':':''}, regex=True)

In [7]:
file3 =open('replaced','r', errors='ignore')
file4 =open('last','w+')
contents2=file3.read()
for x in range(len(contents)):
    if contents2[x] == ":" and contents2[x-1] != "d":
        file4.write("!-!")
    else:
        file4.write(str(contents2[x]))
file4.close()

In [37]:
df2 = pd.read_csv('last', header=None, sep='\n' , encoding='latin-1')
info_only= df2.reset_index()
info_only = info_only[0].str.split(': ', expand=True)


In [38]:
Structure = pd.DataFrame([df["Month"],df["Day"],df["Time"],df["Machine"],df["State"]]) 
Structure=Structure.T

In [42]:
final=pd.concat([Structure ,info_only[1]],axis=1)
final.rename(columns={1:'Info'},index={1:'Info'}, inplace=True)

In [52]:
print(final[["Day","Month","Machine","Info"]])

     Day Month    Machine                                               Info
0     11   Jul  dhcp31371                                    starting etcd=0
Info  11   Jul  dhcp31371                      â etcd.service - Etcd Server
2     11   Jul  dhcp31371                                             Loaded
3     11   Jul  dhcp31371  Active!-! failed (Result!-! start-limit) since...
4     11   Jul  dhcp31371  Main PID!-! 14281 (code=exited!!@@@ status=1/F...
5     11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Fa...
6     11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Un...
7     11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! et...
8     11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! et...
9     11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! St...
10    11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! st...
11    11   Jul  dhcp31371  Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Fa...